<a href="https://colab.research.google.com/github/DavidBillayio/PriceDashboard/blob/main/PriceDashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Price Dashboard

Here we scrape Yahoo Finance for price data and export the data to a csv file.

In [ ]:
# first we import the required modules

from bs4 import BeautifulSoup
import requests
import sched, time, datetime
import pandas as pd

In [ ]:
#Then we define our scheduler class so the program can run automatically
class PeriodicScheduler(object):                                                  
    def __init__(self):                                                           
        self.scheduler = sched.scheduler(time.time, time.sleep)                   
                                                                            
    def setup(self, interval, action, actionargs=()):                             
        action(*actionargs)                                                       
        self.scheduler.enter(interval, 1, self.setup,                             
                        (interval, action, actionargs))                           
                                                                        
    def run(self):                                                                
        self.scheduler.run()

In [ ]:
#Then we will define our various functions that will run our dashboard
def getPrice(link):
    """Scrape the links to get the prices"""
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    price = soup.find_all('div', {'class' :'D(ib) Mend(20px)'})[0].find('span').text
    return price

def periodic(interval, action, actionargs=()):
    """our periodic function to run our PriceRefresh function"""
    sched.scheduler.enter(interval, 1, periodic, priceRefresh, actionargs)
    action(*actionargs)

def priceRefresh(links, update_time,Price_df): 
    """Our priceRefresh function iterates through the links and pulls the prices
        This also creates our update function for the .csv file
    """
    update = []
    update.append(update_time)
    for link in links:
        price = getPrice(links[link])
        print('The value of {} is {}'.format(link, price))
        update.append(price)
    update_df = pd.DataFrame([update])
    return update_df

def periodic_event():
    """This is our base function that iterates each time. 
    We call the priceRefresh function then use the update_df 
    to add to the .csv file. The csv file is overwritten each iteration."""
    global Price_df
    update_time = datetime.datetime.now()
    print(update_time)
    update_df = priceRefresh(sitelinks, update_time,Price_df)
    Price_df = Price_df.append(update_df)
    Price_df.to_csv('PriceData.csv', index = False)
    print('Done')
    

In [ ]:
#Here are the sitelinks. Note, if not using these specific yahoo links the code
#may not work since the scraper looks at the same area of the website each time
# i.e. the price section
sitelinks = {
    'BTCUSD' : 'https://ca.finance.yahoo.com/quote/BTC-USD?p=BTC-USD',
    'ETHUSD' : 'https://ca.finance.yahoo.com/quote/ETH-USD?p=ETH-USD',
    'SP500' : 'https://ca.finance.yahoo.com/quote/%5EGSPC?p=^GSPC',
    'CADUSD' : 'https://ca.finance.yahoo.com/quote/CADUSD=X?p=CADUSD=X',
    'VIX' : 'https://ca.finance.yahoo.com/quote/%5EVIX?p=^VIX'
        }

In [ ]:
#here is an example:
print(getPrice('https://ca.finance.yahoo.com/quote/%5EIXIC?p=^IXIC'))
print(getPrice('https://www.tradingview.com/symbols/NASDAQ-NDAQ/'))

In [ ]:
#After all of that setup we call our scheduler after initializing our Price dataframe
Price_df = pd.DataFrame()

interval = 60 # every minute 
periodic_scheduler = PeriodicScheduler()  
periodic_scheduler.setup(interval, periodic_event) # it executes the event just once  
periodic_scheduler.run() # it starts the scheduler  